<a href="https://colab.research.google.com/github/nishant-pitta/DANN-code/blob/main/MSNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import pickle
import numpy as np
from tqdm import tqdm

# Load MNIST dataset
transform_mnist = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert to 3 channels
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
transform_mnistm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)
train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 34277683.85it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1156294.21it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 9661994.48it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 7221580.28it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
# Load MNIST-M dataset
def load_pickle_file(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f, encoding='latin1')
    return data

mnistm = load_pickle_file('data/mnist-m.pkl')

# Inspect the structure of the loaded data
print(type(mnistm))
print(mnistm.keys())

print(type(mnistm['train']))
print(type(mnistm['test']))

print(mnistm['train'].shape)
print(mnistm['test'].shape)

print(type(mnistm['train'][0]))
print(type(mnistm['test'][0]))

print(mnistm['train'][0])
print(mnistm['test'][0])


<class 'dict'>
dict_keys(['test', 'train'])
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(60000, 28, 28, 3)
(10000, 28, 28, 3)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[[ 60  77 133]
  [ 61  78 134]
  [ 62  79 135]
  ...
  [ 57  76 132]
  [ 57  76 132]
  [ 57  77 130]]

 [[ 60  77 133]
  [ 61  78 134]
  [ 61  78 134]
  ...
  [ 57  76 132]
  [ 57  76 132]
  [ 57  76 132]]

 [[ 61  78 134]
  [ 61  78 134]
  [ 61  78 134]
  ...
  [ 56  75 131]
  [ 56  75 131]
  [ 57  76 132]]

 ...

 [[ 57  76 134]
  [ 57  77 130]
  [ 57  77 130]
  ...
  [  6  34  35]
  [  6  35  33]
  [  7  36  32]]

 [[ 58  77 135]
  [ 58  77 133]
  [ 57  77 130]
  ...
  [  9  37  38]
  [  8  37  35]
  [  8  37  35]]

 [[ 57  73 133]
  [ 62  78 137]
  [ 58  75 131]
  ...
  [  8  37  33]
  [  8  37  35]
  [  8  37  35]]]
[[[ 63  78 133]
  [ 59  76 132]
  [ 65  81 140]
  ...
  [ 15  40  45]
  [  8  37  35]
  [  8  37  35]]

 [[ 62  79 135]
  [ 61  80 136]
  [ 61  80 138]
  ...
  [ 11  35  45]
  [  9  34  38]
  

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import pickle
import numpy as np
from tqdm import tqdm

# Load MNIST dataset
transform_mnist = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert to 3 channels
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
transform_mnistm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)
train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=64, shuffle=False)

# Load MNIST-M dataset
def load_pickle_file(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f, encoding='latin1')
    return data

mnistm = load_pickle_file('data/mnist-m.pkl')
mnistm_train_images = mnistm['train']
mnistm_test_images = mnistm['test']

# Labels are inferred from the MNIST dataset as they are digits from 0 to 9
mnistm_train_labels = np.tile(np.arange(10), 6000)
mnistm_test_labels = np.tile(np.arange(10), 1000)

class MNISTMDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img, target = self.images[idx], self.labels[idx]
        img = img.transpose(2, 0, 1)  # Change to [3, 28, 28]
        img = img.astype(np.float32) / 255.0  # Normalize to [0, 1]
        img = torch.tensor(img)  # Convert to tensor
        if self.transform:
            img = self.transform(img)
        return img, target

mnistm_train_dataset = MNISTMDataset(mnistm_train_images, mnistm_train_labels, transform=None)
mnistm_test_dataset = MNISTMDataset(mnistm_test_images, mnistm_test_labels, transform=None)
mnistm_train_loader = DataLoader(mnistm_train_dataset, batch_size=64, shuffle=True)
mnistm_test_loader = DataLoader(mnistm_test_dataset, batch_size=64, shuffle=False)

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self, input_channels):
        super(CNN, self).__init__()
        # Convolutional and pooling layers
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 48, kernel_size=5, stride=1, padding=2)
        # Fully connected layers
        self.fc1 = nn.Linear(48*7*7, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.pool(self.conv1(x)))
        x = self.relu(self.pool(self.conv2(x)))
        x = x.view(-1, 48*7*7)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
model = CNN(input_channels=3)  # Updated to handle 3-channel inputs
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function (with tqdm progress bar)
def train_model(model, train_loader, criterion, optimizer, num_epochs=1):  # Set num_epochs to 1 for quick testing
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the CNN on MNIST
train_model(model, train_loader, criterion, optimizer)

# Evaluate the CNN on MNIST and MNIST-M
mnist_accuracy = evaluate_model(model, test_loader)
mnistm_accuracy = evaluate_model(model, mnistm_test_loader)


print(f'Test Accuracy on MNIST: {mnist_accuracy:.2f}%')
print(f'Test Accuracy on MNIST-M: {mnistm_accuracy:.2f}%')


Epoch 1/1: 100%|██████████| 938/938 [01:56<00:00,  8.05it/s]


Epoch 1/1, Loss: 0.1512
Test Accuracy on MNIST: 98.51%
Test Accuracy on MNIST-M: 9.61%


In [16]:
import torch.autograd as autograd

class GradientReversalLayer(autograd.Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.alpha, None


In [17]:
class DANN(nn.Module):
    def __init__(self, input_channels):
        super(DANN, self).__init__()
        # Feature extractor (green blocks)
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 48, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Class classifier (purple blocks)
        self.class_classifier = nn.Sequential(
            nn.Linear(48*7*7, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )

        # Domain classifier (dark pink blocks)
        self.domain_classifier = nn.Sequential(
            nn.Linear(48*7*7, 100),
            nn.ReLU(),
            nn.Linear(100, 1),
            nn.Sigmoid()  # Logistic regression
        )

    def forward(self, x, alpha=1.0):
        features = self.feature_extractor(x)
        features = features.view(-1, 48*7*7)
        reverse_features = GradientReversalLayer.apply(features, alpha)
        class_output = self.class_classifier(features)
        domain_output = self.domain_classifier(reverse_features)
        return class_output, domain_output

In [19]:
# Training function for DANN
def train_dann(model, source_loader, target_loader, criterion_class, criterion_domain, optimizer, num_epochs=1, alpha=0.1):
    model.train()
    for epoch in range(num_epochs):
        running_loss_class = 0.0
        running_loss_domain = 0.0
        num_batches = min(len(source_loader), len(target_loader))
        for (source_data, source_labels), (target_data, _) in tqdm(zip(source_loader, target_loader), total=num_batches, desc=f"Epoch {epoch+1}/{num_epochs}"):
            # Combine source and target data
            combined_data = torch.cat((source_data, target_data), 0)
            combined_labels = torch.cat((source_labels, source_labels), 0)  # Dummy labels for target data
            domain_labels = torch.cat((torch.zeros(source_data.size(0)), torch.ones(target_data.size(0))), 0).float().view(-1, 1)  # Ensure proper shape

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            class_output, domain_output = model(combined_data, alpha)

            # Calculate losses
            loss_class = criterion_class(class_output[:source_data.size(0)], source_labels)
            loss_domain = criterion_domain(domain_output, domain_labels)
            loss = loss_class + loss_domain

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss_class += loss_class.item() * source_data.size(0)
            running_loss_domain += loss_domain.item() * combined_data.size(0)

        epoch_loss_class = running_loss_class / len(source_loader.dataset)
        epoch_loss_domain = running_loss_domain / (len(source_loader.dataset) + len(target_loader.dataset))
        print(f'Epoch {epoch+1}/{num_epochs}, Class Loss: {epoch_loss_class:.4f}, Domain Loss: {epoch_loss_domain:.4f}')

# Evaluation function for DANN
def evaluate_dann(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            class_output, _ = model(images)
            _, predicted = torch.max(class_output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Initialize the DANN model
dann_model = DANN(input_channels=3)
criterion_class = nn.CrossEntropyLoss()
criterion_domain = nn.BCELoss()  # Binary Cross-Entropy Loss for domain classifier
optimizer = optim.Adam(dann_model.parameters(), lr=0.001)

# Train the DANN model for one epoch
train_dann(dann_model, train_loader, mnistm_train_loader, criterion_class, criterion_domain, optimizer, num_epochs=1, alpha=0.1)

# Evaluate the DANN model on MNIST and MNIST-M
mnist_accuracy_dann = evaluate_dann(dann_model, test_loader)
mnistm_accuracy_dann = evaluate_dann(dann_model, mnistm_test_loader)
print(f'Test Accuracy on MNIST with DANN: {mnist_accuracy_dann:.2f}%')
print(f'Test Accuracy on MNIST-M with DANN: {mnistm_accuracy_dann:.2f}%')

Epoch 1/1: 100%|██████████| 938/938 [03:32<00:00,  4.41it/s]


Epoch 1/1, Class Loss: 0.2003, Domain Loss: 0.1775
Test Accuracy on MNIST with DANN: 98.41%
Test Accuracy on MNIST-M with DANN: 9.90%
